## **FUZZY SYSTEM AIR QUALITY**
### Metode    : Mamdani
### Defuzz    : Centroid
### Dataset   : AirQualityUCI.xlsx

In [3]:
# 1. IMPORT LIBRARY
import pandas as pd
import numpy as np

In [4]:
# 2. LOAD DATASET
df = pd.read_excel("AirQualityUCI.xlsx")

data = df[['CO(GT)', 'NO2(GT)', 'C6H6(GT)', 'NOx(GT)']].dropna()

In [5]:
# 3. DEFINISI FUNGSI KEANGGOTAAN

def trapezoid(x, a, b, c, d):
    if x <= a or x >= d:
        return 0.0
    elif b <= x <= c:
        return 1.0
    elif a < x < b:
        return (x - a) / (b - a)
    else:
        return (d - x) / (d - c)

def triangle(x, a, b, c):
    if x <= a or x >= c:
        return 0.0
    elif x == b:
        return 1.0
    elif a < x < b:
        return (x - a) / (b - a)
    else:
        return (c - x) / (c - b)

In [6]:
# 4. FUZZIFICATION

def fuzzify(row):
    CO = row['CO(GT)']
    NO2 = row['NO2(GT)']
    O3 = row['C6H6(GT)']   # proxy ozon
    PM10 = row['NOx(GT)'] # proxy polusi

    fuzzy = {}

    # CO
    fuzzy['CO_low'] = trapezoid(CO, 0, 0, 3, 5)
    fuzzy['CO_med'] = triangle(CO, 3, 6, 9)
    fuzzy['CO_high'] = trapezoid(CO, 7, 9, 15, 15)

    # NO2
    fuzzy['NO2_low'] = trapezoid(NO2, 0, 0, 50, 80)
    fuzzy['NO2_med'] = triangle(NO2, 60, 100, 140)
    fuzzy['NO2_high'] = trapezoid(NO2, 120, 150, 200, 200)

    # O3
    fuzzy['O3_low'] = trapezoid(O3, 0, 0, 40, 70)
    fuzzy['O3_med'] = triangle(O3, 50, 100, 150)
    fuzzy['O3_high'] = trapezoid(O3, 130, 160, 200, 200)

    # PM10 (NOx)
    fuzzy['PM10_low'] = trapezoid(PM10, 0, 0, 50, 80)
    fuzzy['PM10_med'] = triangle(PM10, 60, 120, 180)
    fuzzy['PM10_high'] = trapezoid(PM10, 150, 200, 300, 300)

    return fuzzy

In [7]:
# 5. INFERENSI MAMDANI

def inference(fz):
    rules_buruk = []
    rules_sedang = []
    rules_baik = []

    # BURUK
    rules_buruk.append(min(fz['CO_high'], fz['NO2_high'], fz['O3_high'], fz['PM10_high']))
    rules_buruk.append(min(fz['CO_high'], fz['PM10_high']))
    rules_buruk.append(min(fz['NO2_high'], fz['PM10_high']))
    rules_buruk.append(min(fz['CO_high'], fz['O3_high']))
    rules_buruk.append(min(fz['O3_high'], fz['PM10_high']))

    # SEDANG
    rules_sedang.append(min(fz['CO_med'], fz['NO2_med'], fz['PM10_med']))
    rules_sedang.append(min(fz['CO_med'], fz['O3_med']))
    rules_sedang.append(min(fz['NO2_med'], fz['O3_med']))
    rules_sedang.append(min(fz['CO_low'], fz['PM10_med']))
    rules_sedang.append(min(fz['CO_med'], fz['PM10_low']))

    # BAIK
    rules_baik.append(min(fz['CO_low'], fz['NO2_low'], fz['O3_low'], fz['PM10_low']))
    rules_baik.append(min(fz['CO_low'], fz['PM10_low']))
    rules_baik.append(min(fz['NO2_low'], fz['O3_low']))
    rules_baik.append(min(fz['CO_low'], fz['NO2_low'], fz['PM10_med']))
    rules_baik.append(min(fz['CO_med'], fz['NO2_low'], fz['PM10_low']))

    return {
        'buruk': max(rules_buruk),
        'sedang': max(rules_sedang),
        'baik': max(rules_baik)
    }

In [8]:
# 6. DEFUZZIFICATION

def defuzzify(output):
    numerator = 0.0
    denominator = 0.0

    for z in range(0, 101):
        mu_buruk = min(output['buruk'], trapezoid(z, 0, 0, 30, 45))
        mu_sedang = min(output['sedang'], triangle(z, 35, 55, 75))
        mu_baik = min(output['baik'], trapezoid(z, 65, 80, 100, 100))

        mu_agg = max(mu_buruk, mu_sedang, mu_baik)

        numerator += z * mu_agg
        denominator += mu_agg

    if denominator == 0:
        return 0
    return numerator / denominator

In [9]:
# 7. INTERPRETASI OUTPUT

def interpretasi_kualitas(skor):
    """
    Mengubah skor crisp menjadi label linguistik
    """
    if skor < 45:
        return "Buruk"
    elif skor < 75:
        return "Sedang"
    else:
        return "Baik"

In [10]:
# 8. PROSES SEMUA DATA

scores = []

for _, row in data.iterrows():
    fz = fuzzify(row)
    inf = inference(fz)
    score = defuzzify(inf)
    scores.append(score)

data['Skor_Kualitas_Udara'] = scores
data['Label_Kualitas_Udara'] = data['Skor_Kualitas_Udara'].apply(interpretasi_kualitas)

In [11]:
# 9. AMBIL 5 DATA TERBAIK

top5 = data.sort_values(by='Skor_Kualitas_Udara', ascending=False).head(5)

In [12]:
# 10. SIMPAN OUTPUT

top5.to_csv("Top5_Kualitas_Udara_Fuzzy.csv", index=False)

print("✅ Proses selesai")

from IPython.display import display
display(top5.reset_index(drop=True))

✅ Proses selesai


,CO(GT),NO2(GT),C6H6(GT),NOx(GT),Skor_Kualitas_Udara,Label_Kualitas_Udara
0,0.3,46.3,0.952763,60.0,85.654321,Baik
1,0.7,48.0,1.603768,34.0,85.654321,Baik
2,0.7,28.0,1.133431,16.0,85.654321,Baik
3,-200.0,34.0,1.293620,21.0,85.654321,Baik
4,-200.0,42.5,0.801504,52.3,85.654321,Baik
